In [ ]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
from insightface.app import FaceAnalysis
import insightface

Intilize InsightFace

In [ ]:
app = FaceAnalysis(name='buffalo_s') # You can choose anothr model
app.prepare(ctx_id=0, det_size=(640, 640))

Function to load dataset

In [3]:
def load_dataset(dataset_path):
    embeddings = []
    labels = []
    for person_name in os.listdir(dataset_path):
        person_dir = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_dir):
            continue
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            try:
                img = Image.open(image_path).convert('RGB')
                img = np.array(img)
                faces = app.get(img)
                if len(faces) > 0:
                    embedding = faces[0].normed_embedding
                    embeddings.append(embedding)
                    labels.append(person_name)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")
    return np.array(embeddings), labels

Function to recognize face

In [4]:
def recognize_face(embedding, dataset_embeddings, dataset_labels, threshold=0.4):
    if len(dataset_embeddings) == 0:
        return "Unknown", False
    
    similarities = np.dot(dataset_embeddings, embedding) / (np.linalg.norm(dataset_embeddings, axis=1) * np.linalg.norm(embedding))
    max_similarities = np.max(similarities)
    if max_similarities > threshold:
        max_index = np.argmax(similarities)
        return dataset_labels[max_index], True
    return "Unknown", False


Load Dataset

In [ ]:
dataset_path = ""  # path your persons dataset
dataset_embeddings, dataset_labels = load_dataset(dataset_path)
print(f"Loaded {len(dataset_embeddings)} embeddings from dataset")

Intialize webcam

In [6]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Error: could not open webcam")
    exit()

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame")
        break
        
    faces = app.get(frame)
    for face in faces:
        x1, y1, x2, y2 = face.bbox.astype(int)
        
        embedding = face.normed_embedding
        name, access_granted = recognize_face(embedding, dataset_embeddings, dataset_labels)
        
        if access_granted:
            text = f"{name} - Access Granted"
            color = (0, 255, 0)
        else:
            text = f"{name} - Access Denied"
            color = (0, 0, 255)
            
        cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv.putText(frame, text, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
    cv.imshow("Face Recognition", frame)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()